
# EuroPython 2016 session instructions to PDF

# Session Chair Information

## Before your session

1. Plan to be in the talk room at least **10 minutes** before your session.
2. **Double check** that you have the **schedule** with you.
3. Walk to your room.
4. **Check** if there is spare water bottles for the speakers and video adaptors. If something is missing kindly ask the room manager to pick it.
5. Reach for the next speaker.

## For Each Talk

1. Ask the speaker how they would like to be introduced, and double check that you know how to pronounce their name.
2. Ask the speaker about their Q&A preference (see options below) and how they want to be announced of the last 10 or 5 minutes of talk.
3. **Make sure the talk starts on time!** At the talk's scheduled start time, get up on stage, get the microphone, make eye contact with the A/V crew to get their go-ahead, the introduce the speaker.
4. **Watch the clock.** Announce the speaker that their talk is about to finish, they way you agreed.
5. **Two minutes before the talk's scheduled end time**, if it doesn't seem like the speaker is going to stop on their own, quietly get up and stand to the side of the stage. Be ready to interrupt them if they do not finish on time: "I'm very sorry but our time is almost up.", "Can I suggest continuing in an open space?", or "Let's move on to Q&A.".
6. If the speaker requested **Q&A, stand beside the microphone to help moderate** (see section below).
7. When the speaker is finished: start the applause, then stand up and thank them.

## After Your Session

1. Go tell the coordinator or anyone in the reception desk that you finished your session.


## Q&A

Speakers might have different preferences on how they want to Q&A session to be. Please ask them. Here go 3 examples:
1. No questions; they will speak for the duration of their talk then leave.
2. Take a session-chair-moderated questions for the last 5 minutes of the timeslot.


## If You Have Any Problems

If you have any problems, concerns, or questions, try to contact someone on the `europython-volunteers` Telegram channel (http://bit.ly/28QF0FM).

Otherwise you can contact:
1. Christian Barra: barrachris@gmail.com / @christianbarra
2. Alexandre Savio: +34630555357 / alexsavio@gmail.com / @alexsavio
3. TODO Add someone else with a phone number
4. Any conference staff member.


## Moderating Q&A

To moderate a Q&A section, please:

1. Step up to the audience mic and say (something to the effect of): "Thank you {speaker}! We've got a few minutes for questions now".
2. Kindly ask the speaker to repeat the question once they heard it.
3. Questions should be kept short. If the discussion is taking too long (2 minutes), kindly ask to discuss after the talk and reach for the following question, if any.

In [143]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 99999;
//increase max size of output area

<IPython.core.display.Javascript object>

In [144]:
import json
import datetime as dt
import pdfkit as pd
from   operator import itemgetter
from collections import OrderedDict
from operator import itemgetter

from IPython.display import display, HTML
from nbconvert.filters.markdown import markdown2html

In [145]:
talk_sessions = json.load(open('accepted_talks.json'))
talks_admin_url = 'https://ep2016.europython.eu/admin/conference/talk'

In [146]:
show = lambda s: display(HTML(s))

def ordinal(n):
    if 10 <= n % 100 < 20:
        return str(n) + 'th'
    else:
        return  str(n) + {1 : 'st', 2 : 'nd', 3 : 'rd'}.get(n % 10, "th")


def talk_schedule(start, end):
    input_format  = "%Y-%m-%d %H:%M:%S"
    output_format_day = "%A, %B"
    output_format_time = "%H:%M"
    
    output_date = lambda d: "{} {} at {}".format(d.strftime(output_format_day), 
                                                 ordinal(int(d.strftime('%d'))),
                                                 d.strftime(output_format_time))
    start_date = dt.datetime.strptime(start, input_format)
    end_date   = dt.datetime.strptime(end  , input_format)

    return output_date(start_date), output_date(end_date)


def show_talk(talk, show_duration=True, show_link_to_admin=True):
    
    speakers  = talk['speakers']
    title     = talk['title']
    abstract  = talk['abstract_long'][0]
    room      = talk.get('track_title', '').split(', ')[0]
    timerange = talk.get('timerange', '').split(';')[0]
    
    pdf = ''
    pdf += '<h2>{}</h2>\n'.format(title)
    
    if show_link_to_admin:
        pdf += talks_admin_url + '/{}'.format(talk['id'])
        pdf += '<a href={0}>{0}</a>\n'.format(talk_admin_url)
    
    if show_duration:
        pdf += '{} mins.'.format(talk['duration'])
    else:
        pdf += ''

    timerange = talk['timerange'].split(';')[0]
    try:
        start, end = talk_schedule(*timerange.split(', '))
    except:
        start, end = ('', '')

    if start:
        pdf += '<p>'
        pdf += '{} in {}'.format(start, room)
        if show_duration:
            pdf += ' ({})'.format(duration)
        pdf += '</p>\n'

        #show(schedule)
    
    pdf += '<h3><i>{}</i></h2>\n'.format(speakers)
    pdf += '<p>{}</p>\n'.format(markdown2html(abstract))
    pdf += '<br/>\n'
    return pdf

In [147]:
#session_names = ['Keynotes', 'Talks', 'Trainings', 'Help desks',
#                 'EuroPython sessions', 'Other sessions', 'Poster sessions']

session_names = [('Keynote',            'Keynotes'),
                 ('talk',               'Talks'),
                 ('training',           'Tutorials'), 
                 ('poster',             'Posters'),
                 ('helpdesk',           'Helpdesks'),
                 ('EPS session',        'EuroPython Society Sessions'),
                 ('Recruiting session', 'Recruiting'),
                 ('interactive',        'Interactive sessions'),
                 ('Reserved slot',      'TALKS'),
                 ('Lightning talk',     'TALKS'),
                 ('Closing session',    'Closing session'),                 
                ]
session_names = OrderedDict(session_names)

In [148]:
sessions_talks = OrderedDict()

# remove the IDs from the talks
for name, sess in talk_sessions.items():
    sessions_talks[name] = [talk for tid, talk in sess.items()]


# add 'start' time for each talk
for name, talks in sessions_talks.items():
    for talk in talks:
        tr = talk['timerange']
        if not tr:
            talk['start'] = dt.datetime.now()
        else:
            talk['start'] = dt.datetime.strptime(tr.split(',')[0].strip(), "%Y-%m-%d %H:%M:%S")

# sort by
keys = ('start', 'track_title')

for sess, talks in sessions_talks.items():
    sessions_talks[sess] = sorted(talks, key=itemgetter(*keys))
    
# get weekdays
#for talktype, talks in sessions_talks.items():???

In [149]:
for talktype, talks in sessions_talks.items():
    header = ''
    session_name = 'Session {} {} {}'.format(dt.datetime.weekday()) # needs an argment 0o'
    header += 'Session {} {} {}\n'.format(dt.datetime.weekday())
    header += '<h1>{}</h1>\n'.format('Session ')
    
    to_pdf = header
    
    talks = talk_sessions[key] # I don't know where this 'key' came from
    talks = [talks[talk_id] for talk_id in talks]
    talks = sorted(talks, key=itemgetter('title'))
    
    for talk in talks:
        room = talk.get('track_title', '').split(', ')[0]
        to_pdf += show_talk(talk, show_duration=False, show_link_to_admin=False)
        
    pd.from_string(to_pdf, "session_pdfs/"+ session_name +".pdf")

TypeError: descriptor 'weekday' of 'datetime.date' object needs an argument